In [1]:
pip install requests python-dotenv openai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#!/usr/bin/env python3
"""
collect_assets_unstructured.py

- Input: a basic user prompt (unstructured).
- Uses OpenAI to create 4-6 similar queries (fallback if OpenAI not available).
- For each query: searches Pexels (photos + videos) and Freesound (audio).
- Saves combined results to assets_unstructured.json.

Notes:
- Prefer the official APIs (we use them here).
- This is intentionally simple and avoids heavy dependencies.
"""

import os
import time
import json
from typing import List, Dict, Any, Optional
from urllib.parse import quote_plus
from dotenv import load_dotenv
import requests

# OpenAI modern client
from openai import OpenAI

# --------------------------
# Config & keys (from .env)
# --------------------------
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PEXELS_API_KEY = os.getenv("PEXELS_API_KEY")
FREESOUND_API_KEY = os.getenv("FREESOUND_API_KEY")

if not PEXELS_API_KEY:
    raise RuntimeError("Please set PEXELS_API_KEY in your environment or .env file")
if not FREESOUND_API_KEY:
    raise RuntimeError("Please set FREESOUND_API_KEY in your environment or .env file")

USE_OPENAI = bool(OPENAI_API_KEY)
client = None
if USE_OPENAI:
    client = OpenAI(api_key=OPENAI_API_KEY)

# Endpoints & constants
PEXELS_PHOTO_SEARCH = "https://api.pexels.com/v1/search"
PEXELS_VIDEO_SEARCH = "https://api.pexels.com/videos/search"
PEXELS_HEADERS = {"Authorization": PEXELS_API_KEY}

FREESOUND_SEARCH = "https://freesound.org/apiv2/search/text/"
FREESOUND_HEADERS = {"Authorization": f"Token {FREESOUND_API_KEY}"}

# Limits - tune as needed
MAX_QUERIES = 6
PEXELS_PER_QUERY_PHOTOS = 4
PEXELS_PER_QUERY_VIDEOS = 3
FREESOUND_PER_QUERY = 3
DELAY_BETWEEN_PROVIDER_CALLS = 0.25  # polite delay


# --------------------------
# Helper: generate similar queries using OpenAI (or fallback)
# --------------------------
def generate_queries_via_openai(prompt: str, n: int = 5) -> List[str]:
    """
    Ask OpenAI to return a JSON array of short search queries similar to the prompt.
    If OpenAI is not configured or fails, use a conservative fallback.
    """
    if not USE_OPENAI or client is None:
        return fallback_generate_queries(prompt, n)

    system = (
        "You are an assistant that MUST return ONLY a JSON array of short search query strings "
        "(no explanation). Each string should be concise (1-6 words) and be a close variant or related "
        "search term that would help find images/videos/audio for the brief. Return exactly the JSON array."
    )
    user = f"Create {n} concise search queries for this brief: \"{prompt}\". Return JSON array only."

    try:
        resp = client.chat.completions.create(
            model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
            temperature=0.2,
            max_tokens=200,
        )
        # safe access to message content
        try:
            content = resp.choices[0].message["content"]
        except Exception:
            content = resp.choices[0].message.content if hasattr(resp.choices[0].message, "content") else ""
        text = content.strip()
        # strip code fences if present
        if text.startswith("```"):
            lines = text.splitlines()
            if len(lines) >= 3:
                text = "\n".join(lines[1:-1])
        queries = json.loads(text)
        # sanitize and limit
        cleaned = []
        for q in queries:
            if isinstance(q, str):
                s = " ".join(q.split()).strip()
                if s:
                    cleaned.append(s)
            if len(cleaned) >= n:
                break
        if cleaned:
            return cleaned
        else:
            return fallback_generate_queries(prompt, n)
    except Exception as e:
        # fallback
        return fallback_generate_queries(prompt, n)


def fallback_generate_queries(prompt: str, n: int = 5) -> List[str]:
    """
    Conservative deterministic fallback: produce n query variants for the prompt.
    """
    base = prompt.strip()
    tokens = [t for t in base.split() if t]
    queries = []
    # 1: the original brief
    queries.append(base)
    # 2: add synonyms / context phrases
    queries.append(base + " street scene")
    queries.append(base + " crowd")
    queries.append(base + " city life")
    queries.append(base + " people walking at street")
    # dedupe and limit
    final = []
    seen = set()
    for q in queries:
        qn = " ".join(q.split())
        if qn.lower() not in seen:
            seen.add(qn.lower())
            final.append(qn)
        if len(final) >= n:
            break
    return final


# --------------------------
# Pexels API fetch helpers
# --------------------------
def pexels_search_photos(query: str, per_page: int = PEXELS_PER_QUERY_PHOTOS) -> List[Dict[str, Any]]:
    params = {"query": query, "per_page": per_page}
    try:
        r = requests.get(PEXELS_PHOTO_SEARCH, headers=PEXELS_HEADERS, params=params, timeout=12)
        r.raise_for_status()
        data = r.json()
        items = []
        for p in data.get("photos", [])[:per_page]:
            items.append({
                "id": f"photo_{p.get('id')}",
                "url": p.get("src", {}).get("original") or p.get("src", {}).get("large"),
                "width": p.get("width"),
                "height": p.get("height"),
                "photographer": p.get("photographer"),
                "provider": "pexels",
                "raw": p
            })
        return items
    except Exception as e:
        print(f"[PEXELS PHOTOS ERROR] query='{query}' -> {e}")
        return []


def pexels_search_videos(query: str, per_page: int = PEXELS_PER_QUERY_VIDEOS) -> List[Dict[str, Any]]:
    params = {"query": query, "per_page": per_page}
    try:
        r = requests.get(PEXELS_VIDEO_SEARCH, headers=PEXELS_HEADERS, params=params, timeout=12)
        r.raise_for_status()
        data = r.json()
        items = []
        for v in data.get("videos", [])[:per_page]:
            # pick best file (highest width*fps) if available
            files = v.get("video_files", []) or []
            chosen = None
            if files:
                files_sorted = sorted(files, key=lambda f: (f.get("width", 0), f.get("fps", 0)), reverse=True)
                chosen = files_sorted[0]
            items.append({
                "id": f"video_{v.get('id')}",
                "url": (chosen.get("link") if chosen else v.get("url")),
                "duration": v.get("duration"),
                "width": chosen.get("width") if chosen else None,
                "height": chosen.get("height") if chosen else None,
                "provider": "pexels",
                "raw": v
            })
        return items
    except Exception as e:
        print(f"[PEXELS VIDEOS ERROR] query='{query}' -> {e}")
        return []


# --------------------------
# Freesound API fetch helpers
# --------------------------
def freesound_search(query: str, page_size: int = FREESOUND_PER_QUERY) -> List[Dict[str, Any]]:
    params = {"query": query, "page_size": page_size, "fields": "id,name,previews,duration,username,tags,license"}
    try:
        r = requests.get(FREESOUND_SEARCH, headers=FREESOUND_HEADERS, params=params, timeout=12)
        r.raise_for_status()
        data = r.json()
        items = []
        for item in data.get("results", [])[:page_size]:
            previews = item.get("previews", {}) or {}
            preview_url = previews.get("preview-hq-mp3") or previews.get("preview-hq-ogg") or previews.get("preview-lq-mp3")
            items.append({
                "id": f"fs_{item.get('id')}",
                "title": item.get("name"),
                "url": preview_url,
                "duration": item.get("duration"),
                "uploader": item.get("username"),
                "tags": item.get("tags", []),
                "license": item.get("license"),
                "provider": "freesound",
                "raw": item
            })
        return items
    except Exception as e:
        print(f"[FREESOUND ERROR] query='{query}' -> {e}")
        return []


# --------------------------
# Main pipeline
# --------------------------
def collect_assets_for_prompt(prompt: str, num_queries: int = MAX_QUERIES) -> Dict[str, Any]:
    out = {
        "prompt": prompt,
        "generated_queries": [],
        "results": {},  # query -> {pexels: {photos:[], videos:[]}, freesound: {audios:[]}}
        "notes": {}
    }

    # 1) generate queries
    queries = generate_queries_via_openai(prompt, n=num_queries)
    out["generated_queries"] = queries

    # 2) for each query, fetch from Pexels + Freesound
    for q in queries:
        qkey = q
        out["results"][qkey] = {"pexels": {"photos": [], "videos": []}, "freesound": {"audios": []}}
        # Pexels photos
        photos = pexels_search_photos(q)
        time.sleep(DELAY_BETWEEN_PROVIDER_CALLS)
        videos = pexels_search_videos(q)
        time.sleep(DELAY_BETWEEN_PROVIDER_CALLS)
        audios = freesound_search(q)
        time.sleep(DELAY_BETWEEN_PROVIDER_CALLS)
        out["results"][qkey]["pexels"]["photos"] = photos
        out["results"][qkey]["pexels"]["videos"] = videos
        out["results"][qkey]["freesound"]["audios"] = audios

    return out


# --------------------------
# CLI / example usage
# --------------------------
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Collect Pexels + Freesound assets for an unstructured prompt.")
    parser.add_argument("--prompt", "-p", type=str, help="User prompt (e.g. 'bustling city with people')", required=False)
    parser.add_argument("--queries", "-q", type=int, default=5, help="Number of similar queries to generate (default 5)")
    parser.add_argument("--out", "-o", type=str, default="assets_unstructured.json", help="Output JSON filename")
    args,unknown = parser.parse_known_args()

    if not args.prompt:
        # interactive ask
        user_prompt = input("Enter a prompt (e.g. 'bustling city with people'): ").strip()
    else:
        user_prompt = args.prompt.strip()

    print("[RUN] Prompt:", user_prompt)
    plan = collect_assets_for_prompt(user_prompt, num_queries=args.queries)

    # save output
    with open(args.out, "w", encoding="utf-8") as f:
        json.dump(plan, f, indent=2)

    print(f"[DONE] Saved results to {args.out}")
    # print summary
    total_photos = sum(len(plan["results"][q]["pexels"]["photos"]) for q in plan["generated_queries"])
    total_videos = sum(len(plan["results"][q]["pexels"]["videos"]) for q in plan["generated_queries"])
    total_audios = sum(len(plan["results"][q]["freesound"]["audios"]) for q in plan["generated_queries"])
    print(f"Found {total_photos} photos, {total_videos} videos (Pexels), {total_audios} audio previews (Freesound) across {len(plan['generated_queries'])} queries.")


[RUN] Prompt: lady sipping coffee in cafe
[DONE] Saved results to assets_unstructured.json
Found 20 photos, 15 videos (Pexels), 2 audio previews (Freesound) across 5 queries.


In [1]:
import os
import time
import json
from typing import List, Dict, Any
from dotenv import load_dotenv
import requests
from openai import OpenAI

# --------------------------
# Config & keys (from .env)
# --------------------------
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PEXELS_API_KEY = os.getenv("PEXELS_API_KEY")
FREESOUND_API_KEY = os.getenv("FREESOUND_API_KEY")

if not PEXELS_API_KEY:
    raise RuntimeError("Please set PEXELS_API_KEY in your environment or .env file")
if not FREESOUND_API_KEY:
    raise RuntimeError("Please set FREESOUND_API_KEY in your environment or .env file")

USE_OPENAI = bool(OPENAI_API_KEY)
client = OpenAI(api_key=OPENAI_API_KEY) if USE_OPENAI else None

# Endpoints & constants
PEXELS_PHOTO_SEARCH = "https://api.pexels.com/v1/search"
PEXELS_VIDEO_SEARCH = "https://api.pexels.com/videos/search"
PEXELS_HEADERS = {"Authorization": PEXELS_API_KEY}
FREESOUND_SEARCH = "https://freesound.org/apiv2/search/text/"
FREESOUND_HEADERS = {"Authorization": f"Token {FREESOUND_API_KEY}"}

MAX_QUERIES = 6
PEXELS_PER_QUERY_PHOTOS = 4
PEXELS_PER_QUERY_VIDEOS = 3
FREESOUND_PER_QUERY = 3
DELAY_BETWEEN_PROVIDER_CALLS = 0.25


# --------------------------
# Mood Detection
# --------------------------
def detect_mood(text: str) -> str:
    """Use OpenAI to detect mood of a text. Fallback -> Neutral."""
    if not USE_OPENAI or client is None:
        return "Neutral"
    try:
        prompt = f"""
        Analyze the following text and return the overall mood/tone
        in one or two words only (examples: Hopeful, Urgent, Alarming, Inspirational, Calm, Excited).

        Text: "{text}"
        """
        resp = client.chat.completions.create(
            model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
            messages=[{"role": "system", "content": "You are a mood detection assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0.0,
        )
        mood = resp.choices[0].message.content.strip()
        return mood if mood else "Neutral"
    except Exception:
        return "Neutral"


# --------------------------
# Generate queries (OpenAI + fallback)
# --------------------------
def generate_queries_via_openai(prompt: str, n: int = 5) -> List[str]:
    if not USE_OPENAI or client is None:
        return fallback_generate_queries(prompt, n)

    system = (
        "You are an assistant that MUST return ONLY a JSON array of short search query strings "
        "(no explanation). Each string should be concise (1-6 words)."
    )
    user = f"Create {n} concise search queries for this brief: \"{prompt}\". Return JSON array only."

    try:
        resp = client.chat.completions.create(
            model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
            messages=[{"role": "system", "content": system},
                      {"role": "user", "content": user}],
            temperature=0.2,
            max_tokens=200,
        )
        text = resp.choices[0].message.content.strip()
        if text.startswith("```"):
            lines = text.splitlines()
            if len(lines) >= 3:
                text = "\n".join(lines[1:-1])
        queries = json.loads(text)
        return [q.strip() for q in queries if isinstance(q, str)][:n]
    except Exception:
        return fallback_generate_queries(prompt, n)


def fallback_generate_queries(prompt: str, n: int = 5) -> List[str]:
    base = prompt.strip()
    queries = [
        base,
        base + " street scene",
        base + " crowd",
        base + " city life",
        base + " people walking at street",
    ]
    return queries[:n]


# --------------------------
# Pexels + Freesound search
# --------------------------
def pexels_search_photos(query: str, per_page: int = PEXELS_PER_QUERY_PHOTOS) -> List[Dict[str, Any]]:
    params = {"query": query, "per_page": per_page}
    try:
        r = requests.get(PEXELS_PHOTO_SEARCH, headers=PEXELS_HEADERS, params=params, timeout=12)
        r.raise_for_status()
        return [{
            "id": f"photo_{p['id']}",
            "url": p.get("src", {}).get("original") or p.get("src", {}).get("large"),
            "width": p.get("width"),
            "height": p.get("height"),
            "photographer": p.get("photographer"),
            "provider": "pexels"
        } for p in r.json().get("photos", [])[:per_page]]
    except Exception as e:
        print(f"[PEXELS PHOTOS ERROR] {e}")
        return []


def pexels_search_videos(query: str, per_page: int = PEXELS_PER_QUERY_VIDEOS) -> List[Dict[str, Any]]:
    params = {"query": query, "per_page": per_page}
    try:
        r = requests.get(PEXELS_VIDEO_SEARCH, headers=PEXELS_HEADERS, params=params, timeout=12)
        r.raise_for_status()
        items = []
        for v in r.json().get("videos", [])[:per_page]:
            files = v.get("video_files", [])
            chosen = sorted(files, key=lambda f: (f.get("width", 0), f.get("fps", 0)), reverse=True)[0] if files else None
            items.append({
                "id": f"video_{v['id']}",
                "url": chosen.get("link") if chosen else v.get("url"),
                "duration": v.get("duration"),
                "width": chosen.get("width") if chosen else None,
                "height": chosen.get("height") if chosen else None,
                "provider": "pexels"
            })
        return items
    except Exception as e:
        print(f"[PEXELS VIDEOS ERROR] {e}")
        return []


def freesound_search(query: str, page_size: int = FREESOUND_PER_QUERY) -> List[Dict[str, Any]]:
    params = {"query": query, "page_size": page_size, "fields": "id,name,previews,duration,username,tags,license"}
    try:
        r = requests.get(FREESOUND_SEARCH, headers=FREESOUND_HEADERS, params=params, timeout=12)
        r.raise_for_status()
        return [{
            "id": f"fs_{item['id']}",
            "title": item["name"],
            "url": item.get("previews", {}).get("preview-hq-mp3"),
            "duration": item["duration"],
            "uploader": item["username"],
            "tags": item.get("tags", []),
            "license": item["license"],
            "provider": "freesound"
        } for item in r.json().get("results", [])[:page_size]]
    except Exception as e:
        print(f"[FREESOUND ERROR] {e}")
        return []


# --------------------------
# Main pipeline
# --------------------------
def collect_assets_for_prompt(prompt: str, num_queries: int = MAX_QUERIES) -> Dict[str, Any]:
    out = {
        "prompt": prompt,
        "global_mood": detect_mood(prompt),
        "generated_queries": [],
        "results": {}
    }

    queries = generate_queries_via_openai(prompt, n=num_queries)
    out["generated_queries"] = queries

    for q in queries:
        out["results"][q] = {
            "query_mood": detect_mood(q),
            "pexels": {"photos": [], "videos": []},
            "freesound": {"audios": []}
        }
        out["results"][q]["pexels"]["photos"] = pexels_search_photos(q)
        time.sleep(DELAY_BETWEEN_PROVIDER_CALLS)
        out["results"][q]["pexels"]["videos"] = pexels_search_videos(q)
        time.sleep(DELAY_BETWEEN_PROVIDER_CALLS)
        out["results"][q]["freesound"]["audios"] = freesound_search(q)
        time.sleep(DELAY_BETWEEN_PROVIDER_CALLS)

    return out


# --------------------------
# CLI / example usage
# --------------------------
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description="Collect Pexels + Freesound assets with mood detection.")
    parser.add_argument("--prompt", "-p", type=str, help="User prompt", required=False)
    parser.add_argument("--queries", "-q", type=int, default=5, help="Number of queries")
    parser.add_argument("--out", "-o", type=str, default="assets_with_mood.json", help="Output JSON filename")
    args, _ = parser.parse_known_args()

    user_prompt = args.prompt.strip() if args.prompt else input("Enter a prompt: ").strip()

    print("[RUN] Prompt:", user_prompt)
    plan = collect_assets_for_prompt(user_prompt, num_queries=args.queries)

    with open(args.out, "w", encoding="utf-8") as f:
        json.dump(plan, f, indent=2)

    print(f"[DONE] Saved results to {args.out}")
    total_photos = sum(len(plan["results"][q]["pexels"]["photos"]) for q in plan["generated_queries"])
    total_videos = sum(len(plan["results"][q]["pexels"]["videos"]) for q in plan["generated_queries"])
    total_audios = sum(len(plan["results"][q]["freesound"]["audios"]) for q in plan["generated_queries"])
    print(f"Found {total_photos} photos, {total_videos} videos, {total_audios} audios across {len(plan['generated_queries'])} queries.")


[RUN] Prompt: a cat walking on a wall and looking for a mouse
[DONE] Saved results to assets_with_mood.json
Found 20 photos, 15 videos, 0 audios across 5 queries.
